# Панельная регрессия с Linearmodels. Тестирование гипотез

In [ ]:
import numpy as np
import pandas as pd

from linearmodels import PooledOLS     # Pool модель
from linearmodels import RandomEffects # RE-модель
from linearmodels import PanelOLS      # FE-модель

from linearmodels.panel import compare # несколько регрессий в одной таблице

## Загрузка и подготовка датасета

In [ ]:
df = pd.read_csv('Guns.csv')
df

In [ ]:
# зададим панельную структуру через мульти-индекс
# state - индивидуальный индекс, year - временной индекс
panel_df = df.set_index(['state', 'year'])
panel_df

## Панельные регрессии
Рассмотрим модель `log(violent)` на `law, log(income), log(population), log(prisoners), log(robbery), log(murder), male, density`  в трёх  вариантах:
- Pool
- Random effects
- Fixed effects

Т.к. формула спецификации одна и та же, то зададим её в виде строковой переменной (не забываем указать 1 в начале)

Модели подгоним с разными оценками ковариационной матрицы для коэффициентов и сравним результаты

In [ ]:
spec = 'np.log(violent)~1+law+np.log(income)+np.log(population)+np.log(prisoners)+np.log(robbery)+np.log(murder)+male+density'

In [ ]:
mod_pl = PooledOLS.from_formula(formula=spec, data=panel_df)

mod_re = RandomEffects.from_formula(formula=spec, data=panel_df)

mod_fe =PanelOLS.from_formula(formula=spec, data=panel_df, drop_absorbed=True)

# подгонка Pool с разными оценками ковариационной матрицы: OLS, Arellano-Bond, Driscoll-Kraay
res_pl_ols = mod_pl.fit()
res_pl_AB = mod_pl.fit(cov_type='clustered', cluster_entity=True)
res_pl_DK = mod_pl.fit(cov_type='kernel')

# подгонка RE с разными оценками ковариационной матрицы: OLS, Arellano-Bond, Driscoll-Kraay
res_re_ols = mod_re.fit()
res_re_AB = mod_re.fit(cov_type='clustered', cluster_entity=True)
res_re_DK = mod_re.fit(cov_type='kernel')

# подгонка RE с разными оценками ковариационной матрицы: OLS, Arellano-Bond, Driscoll-Kraay
res_fe_ols = mod_fe.fit()
res_fe_AB = mod_fe.fit(cov_type='clustered', cluster_entity=True)
res_fe_DK = mod_fe.fit(cov_type='kernel')

In [ ]:
# PL, RE, FE-модели с оценкой ковариационной матрицы по умолчанию
res_dict = {'PL': res_pl_ols, 'RE':res_re_ols, 'FE':res_fe_ols}

compare(results=res_dict, precision='std-errors', stars=True)

In [ ]:

# RE & FE - модели с разными оценками ковариационной матрицы
res_dict = {'RE(OLS)': res_re_ols, 'RE(AB)':res_re_AB, 'RE(DK)':res_re_DK, 'FE(OLS)': res_fe_ols, 'FE(AB)':res_fe_AB, 'FE(DK)':res_fe_DK}

compare(results=res_dict, precision='std-errors', stars=True)